In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/feature_engineered_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFIFA version distribution:")
print(df['fifa_version'].value_counts().sort_index())
print(f"\nTarget distribution:")
print(df['big_potential'].value_counts())

Dataset shape: (41458, 29)

FIFA version distribution:
fifa_version
17.0    8889
18.0    8814
19.0    8777
20.0    8091
21.0    6887
Name: count, dtype: int64

Target distribution:
big_potential
0    31162
1    10296
Name: count, dtype: int64


In [2]:
# Define temporal splits
train_versions = [17.0, 18.0, 19.0,20.0]
val_version = 21.0
test_version = 21.0

# Split data by FIFA version
df_train = df[df['fifa_version'].isin(train_versions)]
df_val = df[df['fifa_version'] == val_version]
df_test = df[df['fifa_version'] == test_version]

print(f"Train (FIFA 17-20): {len(df_train)} samples")
print(f"Validation (FIFA 21): {len(df_val)} samples")
print(f"Test (FIFA 21): {len(df_test)} samples")

Train (FIFA 17-20): 34571 samples
Validation (FIFA 21): 6887 samples
Test (FIFA 21): 6887 samples


In [3]:
# All columns except fifa_version and big_potential are features
feature_columns = [col for col in df.columns if col not in ['fifa_version', 'big_potential']]
target = 'big_potential'

print(f"Number of features: {len(feature_columns)}")
print(f"Features: {feature_columns}")

# Prepare datasets
X_train = df_train[feature_columns]
y_train = df_train[target]

X_val = df_val[feature_columns]
y_val = df_val[target]

X_test = df_test[feature_columns]
y_test = df_test[target]

Number of features: 27
Features: ['age', 'physic', 'mentality_aggression', 'mentality_interceptions', 'power_stamina', 'power_strength', 'defending_marking_awareness', 'power_jumping', 'defending_standing_tackle', 'defending_sliding_tackle', 'attacking_heading_accuracy', 'mentality_composure', 'movement_reactions', 'skill_long_passing', 'skill_dribbling', 'skill_fk_accuracy', 'skill_ball_control', 'attacking_crossing', 'power_shot_power', 'attacking_finishing', 'skill_curve', 'movement_balance', 'attacking_volleys', 'power_long_shots', 'mentality_vision', 'mentality_penalties', 'movement_agility']


In [4]:
X_train

,age,physic,mentality_aggression,mentality_interceptions,power_stamina,power_strength,defending_marking_awareness,power_jumping,defending_standing_tackle,defending_sliding_tackle,...,attacking_crossing,power_shot_power,attacking_finishing,skill_curve,movement_balance,attacking_volleys,power_long_shots,mentality_vision,mentality_penalties,movement_agility
0,32,60.0,58,68,64,59,57,54,57,56,...,79,65,73,80,86,74,74,94,71,82
1,33,58.0,58,66,58,58,57,52,57,56,...,77,65,70,80,84,74,71,94,71,79
2,34,57.0,58,66,55,58,67,47,57,56,...,77,65,70,80,84,74,71,94,71,79
3,35,66.0,87,68,51,64,64,72,66,65,...,78,79,68,81,84,69,77,77,85,66
4,31,66.0,70,81,55,69,80,70,80,76,...,34,68,20,49,63,19,40,46,35,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40815,18,49.0,51,61,60,43,58,55,63,58,...,60,26,23,33,77,28,23,37,39,59
40817,20,70.0,74,57,65,71,58,61,64,62,...,52,42,21,28,54,23,24,42,30,58
40819,18,51.0,32,42,59,53,40,61,43,42,...,39,44,41,35,52,31,39,47,41,65
40821,17,56.0,55,47,61,54,36,59,38,40,...,30,46,28,36,57,31,30,47,39,59


In [5]:

from sklearn.model_selection import train_test_split
# Stratified split: 20% labeled, 80% unlabeled from training data
X_labeled, X_unlabeled, y_labeled, y_unlabeled_true = train_test_split(
    X_train, y_train, 
    test_size=0.8,  # 80% unlabeled
    stratify=y_train, 
    random_state=42
)

# Create unlabeled labels (-1 for semi-supervised algorithms)
y_unlabeled = np.full(len(y_unlabeled_true), -1)

# Combine for semi-supervised training
X_train_ssl = np.vstack([X_labeled.values, X_unlabeled.values])
y_train_ssl = np.concatenate([y_labeled.values, y_unlabeled])

print(f"Labeled samples: {len(y_labeled)} ({len(y_labeled)/len(y_train)*100:.1f}%)")
print(f"Unlabeled samples: {len(y_unlabeled)} ({len(y_unlabeled)/len(y_train)*100:.1f}%)")

Labeled samples: 6914 (20.0%)
Unlabeled samples: 27657 (80.0%)


In [6]:
X_unlabeled

,age,physic,mentality_aggression,mentality_interceptions,power_stamina,power_strength,defending_marking_awareness,power_jumping,defending_standing_tackle,defending_sliding_tackle,...,attacking_crossing,power_shot_power,attacking_finishing,skill_curve,movement_balance,attacking_volleys,power_long_shots,mentality_vision,mentality_penalties,movement_agility
13619,19,63.0,62,12,67,62,14,63,13,13,...,62,67,58,65,73,58,61,47,62,80
497,32,70.0,75,78,75,66,75,55,76,74,...,88,75,64,86,76,71,69,76,60,77
13733,24,67.0,55,65,68,71,67,77,67,68,...,33,49,30,33,70,32,24,38,50,69
4519,29,63.0,56,49,70,62,38,62,53,40,...,73,83,76,52,83,82,83,79,72,89
20132,29,74.0,61,42,80,74,25,90,40,42,...,70,80,75,71,76,65,74,67,70,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15696,24,73.0,74,63,80,69,64,74,65,63,...,65,63,54,61,81,44,61,64,56,76
6812,26,79.0,86,68,79,76,66,79,74,72,...,68,73,55,45,68,33,66,62,60,66
12918,27,66.0,34,19,74,73,23,90,28,21,...,55,59,68,41,89,45,45,62,62,83
11526,28,70.0,79,62,85,59,64,77,62,66,...,73,75,50,67,77,61,64,77,51,83


In [7]:
y_unlabeled

array([-1, -1, -1, ..., -1, -1, -1], shape=(27657,))

In [8]:
print(type(y_train_ssl))
print(type(X_train_ssl))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
# converting to numpy array
X_val_array = X_val.values if hasattr(X_val, 'values') else X_val
print(type(X_val_array))


<class 'numpy.ndarray'>


In [10]:
#apply z-score scaling 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_ssl_scaled = scaler.fit_transform(X_train_ssl)
X_val_array_scaled = scaler.transform(X_val_array)
X_labeled_scaled = scaler.transform(X_labeled.to_numpy())



In [11]:
print("Class distribution across splits:")
print(f"Train total: {y_train.value_counts().to_dict()}")
print(f"Labeled only: {y_labeled.value_counts().to_dict()}")
print(f"Validation: {y_val.value_counts().to_dict()}")
print(f"Test: {y_test.value_counts().to_dict()}")

Class distribution across splits:
Train total: {0: 25968, 1: 8603}
Labeled only: {0: 5193, 1: 1721}
Validation: {0: 5194, 1: 1693}
Test: {0: 5194, 1: 1693}


In [ ]:

from xgboost import XGBClassifier as xgb
#baseline = xgb(
 #   n_estimators=300,
  #  max_depth=4,
   # learning_rate=0.05,
    #subsample=0.8,
    #colsample_bytree=0.8,
    #reg_alpha=0.1,
    #reg_lambda=1.0,
    #scale_pos_weight=3,    
    #eval_metric="logloss",
    #tree_method="hist",
    #random_state=42
#)
baseline=xgb(
   n_estimators=250,      # Balanced number of trees
    max_depth=3,            # Shallower to prevent overfitting
    learning_rate=0.03,     # Conservative learning rate
    subsample=0.7,          # Aggressive subsampling
    colsample_bytree=0.7,
    reg_alpha=0.5,          # Higher regularization
    reg_lambda=2.0,
    scale_pos_weight=3,    
    eval_metric="logloss",
    tree_method="hist",
    random_state=42
)
baseline.fit(X_labeled, y_labeled)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [22]:
# Evaluate baseline model on validation set (FIFA 20)
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    f1_score, roc_auc_score, accuracy_score,
    precision_score, recall_score
)

# Predictions on validation set
y_val_pred = baseline.predict(X_val)
y_val_proba = baseline.predict_proba(X_val)[:, 1]

# Calculate metrics
print("="*60)
print("BASELINE MODEL - Validation Set (FIFA 21) Metrics")
print("="*60)
print(f"\nAccuracy:  {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.4f}")
print(f"Recall:    {recall_score(y_val, y_val_pred):.4f}")
print(f"F1 Score:  {f1_score(y_val, y_val_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_val, y_val_proba):.4f}")

print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_val, y_val_pred, target_names=['No Big Potential', 'Big Potential']))

print("-"*60)
print("Confusion Matrix:")
print("-"*60)
cm = confusion_matrix(y_val, y_val_pred)
print(f"                  Predicted")
print(f"                  No BP    BP")
print(f"Actual No BP     {cm[0,0]:5d}  {cm[0,1]:5d}")
print(f"Actual BP        {cm[1,0]:5d}  {cm[1,1]:5d}")


BASELINE MODEL - Validation Set (FIFA 21) Metrics

Accuracy:  0.6640
Precision: 0.3929
Recall:    0.6728
F1 Score:  0.4961
ROC-AUC:   0.7278

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
                  precision    recall  f1-score   support

No Big Potential       0.86      0.66      0.75      5194
   Big Potential       0.39      0.67      0.50      1693

        accuracy                           0.66      6887
       macro avg       0.63      0.67      0.62      6887
    weighted avg       0.75      0.66      0.69      6887

------------------------------------------------------------
Confusion Matrix:
------------------------------------------------------------
                  Predicted
                  No BP    BP
Actual No BP      3434   1760
Actual BP          554   1139


In [27]:
# Evaluate self-training model on validation set (FIFA 21)
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    f1_score, roc_auc_score, accuracy_score,
    precision_score, recall_score
)

#self-Training classifier
from sklearn.semi_supervised import SelfTrainingClassifier


 # self_training = SelfTrainingClassifier(
  # RandomForestClassifier(
   # n_estimators=200,
   # max_depth=10,
   # min_samples_split=5,
   # min_samples_leaf=4,
   # max_features="sqrt",
   # class_weight="balanced",
   # random_state=42,
   # n_jobs=-1
   # ),
   # threshold=0.9,
   # max_iter=3
#) 
xgb_baseline = xgb(
    n_estimators=250,      # Balanced number of trees
    max_depth=3,            # Shallower to prevent overfitting
    learning_rate=0.03,     # Conservative learning rate
    subsample=0.7,          # Aggressive subsampling
    colsample_bytree=0.7,
    reg_alpha=0.5,          # Higher regularization
    reg_lambda=2.0,
    scale_pos_weight=3,    
    eval_metric="logloss",
    tree_method="hist",
    random_state=42
)

self_training = SelfTrainingClassifier(
    xgb_baseline,
    threshold=0.9,     
    max_iter=2,
    verbose=1
)
self_training.fit(X_train_ssl_scaled, y_train_ssl)

# Predictions on validation set
y_ssl_pred = self_training.predict(X_val_array_scaled)
y_ssl_proba = self_training.predict_proba(X_val_array_scaled)[:, 1]

# Calculate metrics
print("="*60)
print("SELF-TRAINING MODEL - Validation Set (FIFA 21) Metrics")
print("="*60)
print(f"\nAccuracy:  {accuracy_score(y_val, y_ssl_pred):.4f}")
print(f"Precision: {precision_score(y_val, y_ssl_pred):.4f}")
print(f"Recall:    {recall_score(y_val, y_ssl_pred):.4f}")
print(f"F1 Score:  {f1_score(y_val, y_ssl_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_val, y_ssl_proba):.4f}")

print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_val, y_ssl_pred, target_names=['No Big Potential', 'Big Potential']))

print("-"*60)
print("Confusion Matrix:")
print("-"*60)
cm = confusion_matrix(y_val, y_ssl_pred)
print(f"                  Predicted")
print(f"                  No BP    BP")
print(f"Actual No BP     {cm[0,0]:5d}  {cm[0,1]:5d}")
print(f"Actual BP        {cm[1,0]:5d}  {cm[1,1]:5d}")



End of iteration 1, added 668 new labels.
End of iteration 2, added 820 new labels.
SELF-TRAINING MODEL - Validation Set (FIFA 21) Metrics

Accuracy:  0.6623
Precision: 0.3927
Recall:    0.6846
F1 Score:  0.4991
ROC-AUC:   0.7238

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
                  precision    recall  f1-score   support

No Big Potential       0.86      0.65      0.75      5194
   Big Potential       0.39      0.68      0.50      1693

        accuracy                           0.66      6887
       macro avg       0.63      0.67      0.62      6887
    weighted avg       0.75      0.66      0.68      6887

------------------------------------------------------------
Confusion Matrix:
------------------------------------------------------------
                  Predicted
                  No BP    BP
Actual No BP      3402   1792
Actual BP          534   1159


In [19]:
y_labeled_pred = self_training.predict(X_labeled_scaled)
y_labeled_proba = self_training.predict_proba(X_labeled_scaled)[:, 1]

# Calculate metrics on original labeled data
print("="*60)
print("SELF-TRAINING MODEL - Training Set Metrics (Original Labeled Data)")
print("="*60)
print(f"\nLabeled samples: {len(y_labeled)}")
print(f"\nAccuracy:  {accuracy_score(y_labeled, y_labeled_pred):.4f}")
print(f"Precision: {precision_score(y_labeled, y_labeled_pred):.4f}")
print(f"Recall:    {recall_score(y_labeled, y_labeled_pred):.4f}")
print(f"F1 Score:  {f1_score(y_labeled, y_labeled_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_labeled, y_labeled_proba):.4f}")

print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_labeled, y_labeled_pred, target_names=['No Big Potential', 'Big Potential']))

print("-"*60)
print("Confusion Matrix:")
print("-"*60)
cm = confusion_matrix(y_labeled, y_labeled_pred)
print(f"                  Predicted")
print(f"                  No BP    BP")
print(f"Actual No BP     {cm[0,0]:5d}  {cm[0,1]:5d}")
print(f"Actual BP        {cm[1,0]:5d}  {cm[1,1]:5d}")

SELF-TRAINING MODEL - Training Set Metrics (Original Labeled Data)

Labeled samples: 6914

Accuracy:  0.6955
Precision: 0.4351
Recall:    0.7484
F1 Score:  0.5503
ROC-AUC:   0.7886

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
                  precision    recall  f1-score   support

No Big Potential       0.89      0.68      0.77      5193
   Big Potential       0.44      0.75      0.55      1721

        accuracy                           0.70      6914
       macro avg       0.66      0.71      0.66      6914
    weighted avg       0.78      0.70      0.72      6914

------------------------------------------------------------
Confusion Matrix:
------------------------------------------------------------
                  Predicted
                  No BP    BP
Actual No BP      3521   1672
Actual BP          433   1288
